In [ ]:
from SPARQLWrapper import SPARQLWrapper
import pandas as pd
from sqlalchemy import create_engine

query_companies = """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?org {{
        SELECT DISTINCT ?org {{
            ?org a/rdfs:subClassOf* dbo:Company .
        }} ORDER BY ?org
    }}
    LIMIT {limit} OFFSET {offset}
"""
base_url = 'http://dbpedia.org/resource/'
sparql = SPARQLWrapper("http://dbpedia.org/sparql", returnFormat='csv')
engine = create_engine('sqlite:///data/training_data.db')
limit = 10000
offset = 0
iteration = 1

while limit > 0:
    sparql.setQuery(query_companies.format(limit=limit, offset=offset))
    try:
        print(f"Loading comapanies from {offset+1} to {offset+limit}")
        ret = sparql.query().convert().decode().replace("\"", "").split('\n')
        print("Parsing...")
        ret = [l.replace(base_url, '') for l in ret if base_url in l]
        if len(ret) > 0:
            print("Storing into db")
            df = pd.DataFrame(ret, columns=['name'])
            df['iteration'] = iteration
            df['source'] = 'dbp'
            print("Data frame shape:", df.shape)
            df.to_sql('company_names', con=engine, if_exists='append')
            offset += limit
            iteration += 1
        else:
            limit = -1
            print("Done!")
    except Exception as e:
        limit = limit // 10
        print(e)

In [ ]:
import wikipedia

page = wikipedia.page('&pizza', auto_suggest=False)